In [1]:
from Grafo import Grafo
import numpy as np

grafito = Grafo()

grafito.agregar_nodo("pepe")
grafito.agregar_nodo("luis")
grafito.agregar_nodo("solito")
grafito.agregar_arista("pepe","luis",3)
grafito.agregar_arista("luis","pepe",-3)
print(grafito)

grafito.agregar_nodo("amiguis1")
grafito.agregar_nodo("amiguis2")
grafito.agregar_arista("amiguis1","amiguis2",no_dirigido=True)
print(grafito)

{'pepe': 0, 'luis': 1, 'solito': 2}
<Nodos: ['pepe', 'luis', 'solito']
 Adyacencias:
[[ 0.  3.  0.]
 [-3.  0.  0.]
 [ 0.  0.  0.]] >
{'pepe': 0, 'luis': 1, 'solito': 2, 'amiguis1': 3, 'amiguis2': 4}
<Nodos: ['pepe', 'luis', 'solito', 'amiguis1', 'amiguis2']
 Adyacencias:
[[ 0.  3.  0.  0.  0.]
 [-3.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.]] >


In [22]:
grafito._indices.items()

dict_items([('pepe', 0), ('luis', 1), ('solito', 2), ('amiguis1', 3), ('amiguis2', 4)])

In [21]:
indices = (np.where(grafito._mat_ady[0,:].todense()!=0))[1]
[nombre for nombre,idx in grafito._indices.items() if idx in indices]


['luis']

In [ ]:
x

In [ ]:
m={"pepe":1, "gaston":4}
m["gaston"]

In [ ]:
m["popo"]=-1

In [ ]:
m

In [ ]:
list(m.values())

In [ ]:
0.0==0

In [ ]:
ts=grafito._mat_ady[:,0]

In [ ]:
ts

In [ ]:
grafito._mat_ady[:,0]=0

In [ ]:
grafito._mat_ady.todense()

In [9]:
li=[1,9,3,4]
del li[0]
li.remove(9)
li

[3, 4]

In [10]:
li.index(4)

1

In [12]:
li.extend([10,11])

In [13]:
11 in li

True

In [25]:
pepe = {"vero": "poshito rosa", "nando": 2, "poshito": 3}

In [26]:
valid_dict = {(1,2,3):[1,2,3]} 
valid_dict #debe ser inmutable

{(1, 2, 3): [1, 2, 3]}

In [27]:
list(pepe.keys())

['vero', 'nando', 'poshito']

In [28]:
list(pepe.values())

['poshito rosa', 2, 3]

In [23]:
"vero" in pepe

True

In [29]:
pepe.get("vero")

'poshito rosa'

In [30]:
pepe.update({"cone":"comida"})

In [31]:
pepe

{'cone': 'comida', 'nando': 2, 'poshito': 3, 'vero': 'poshito rosa'}

In [32]:
pepe

{'cone': 'comida', 'nando': 2, 'poshito': 3, 'vero': 'poshito rosa'}

In [34]:
pepe["platano"] = "s1 sen10r"

In [35]:
pepe

{'cone': 'comida',
 'nando': 2,
 'platano': 's1 sen10r',
 'poshito': 3,
 'vero': 'poshito rosa'}

In [36]:
del pepe["platano"]

In [37]:
pepe

{'cone': 'comida', 'nando': 2, 'poshito': 3, 'vero': 'poshito rosa'}

In [43]:
primer_set={2,3}
other_set = {3, 4, 5, 6}
primer_set & other_set  # da solo {3}
primer_set | other_set #es la union sin dos veces la interseccion digamos
primer_set - other_set #este hace algo raro, saca los elementos de la primera que esten en la segunda

{2}

In [44]:
{1, 2, 3, 4} ^ {2, 3, 5} #este dice cuales son distintos entre ambos

{1, 4, 5}

In [46]:
# Check if set on the left is a subset of set on the right
{1, 2} <= {1, 2, 3} #chequea si lo primero es un subespacio generado 
#por el segundo, con el piquito para el otro lado chequea si el de la derecha lo es

True

In [47]:
2 in primer_set

True